In [ ]:
import os
import numpy as np
from skimage import io, transform, color
from skimage.feature import local_binary_pattern, hog
from skimage.filters import gabor_kernel
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Concatenate
from keras.models import Model

# Define function to extract LBP features from an image
def extract_lbp_features(image):
    # Compute LBP features
    lbp_radius = 3
    lbp_n_points = 8 * lbp_radius
    lbp_method = 'uniform'
    lbp_image = local_binary_pattern(image, lbp_n_points, lbp_radius, method=lbp_method)
    lbp_histogram, _ = np.histogram(lbp_image, bins=np.arange(0, lbp_n_points + 3), range=(0, lbp_n_points + 2), density=True)
    lbp_features = lbp_histogram

    return lbp_features

# Define function to extract HOG features from an image
def extract_hog_features(image):
    # Compute HOG features
    hog_orientation = 9
    hog_pixels_per_cell = (8, 8)
    hog_cells_per_block = (3, 3)
    hog_block_norm = 'L2-Hys'
    hog_features = hog(image, orientations=hog_orientation, pixels_per_cell=hog_pixels_per_cell, cells_per_block=hog_cells_per_block, block_norm=hog_block_norm)

    return hog_features

# Define function to extract Gabor features from an image
def extract_gabor_features(image):
    # Compute Gabor features
    gabor_frequencies = [0.1, 0.2, 0.3]
    gabor_thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    gabor_features = []
    for frequency in gabor_frequencies:
        for theta in gabor_thetas:
            gabor_kernel_real, gabor_kernel_imag = gabor_kernel(frequency, theta=theta)
            gabor_image_real = np.abs(convolve(image, gabor_kernel_real, mode='same'))
            gabor_image_imag = np.abs(convolve(image, gabor_kernel_imag, mode='same'))
            gabor_image = gabor_image_real + gabor_image_imag
            gabor_features.append(np.mean(gabor_image))
    gabor_features = np.array(gabor_features)

    return gabor_features

# Define function to load images and extract features
def load_images(folder):
    image_filenames = os.listdir(folder)
    images = []
    labels = []
    for filename in image_filenames:
        label = int(filename.split('_')[0])
        image = io.imread(os.path.join(folder, filename))
        image = transform.resize(image, (48, 48))
        gray_image = color.rgb2gray(image)
        
        # extract features from the image
        lbp_features = extract_lbp_features(gray_image)
        hog_features = extract_hog_features(gray_image)
        gabor_features = extract_gabor_features(gray_image)
        
        # concatenate the features and original image data
        features = np.concatenate((lbp_features, hog_features, gabor_features))
        images.append(image)
        labels.append(label)
        
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels, features

# Load training data and extract features
train_folder = 'train'
train_images, train_labels, train_features = load_images(train_folder)

# Load testing data and extract features
test_folder = 'test'
test_images, test_labels, test_features = load_images(test


In [ ]:
# Load training data and extract features
train_folder = 'train'
train_images, train_labels, train_features = load_images(train_folder)

# Load testing data and extract features
test_folder = 'test'
test_images, test_labels, test_features = load_images(test_folder)

# Define the neural network architecture
input_layer = Input(shape=(48, 48, 3))
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)

lbp_input_layer = Input(shape=(26,))
hog_input_layer = Input(shape=(3780,))
gabor_input_layer = Input(shape=(32,))

concat_layer = concatenate([x, lbp_input_layer, hog_input_layer, gabor_input_layer])
hidden_layer = Dense(512, activation='relu')(concat_layer)
output_layer = Dense(7, activation='softmax')(hidden_layer)

model = Model(inputs=[input_layer, lbp_input_layer, hog_input_layer, gabor_input_layer], outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
